# Data-QA for event.mediawiki_edit_attempt
Nov 1, 2022 [task T320281](https://phabricator.wikimedia.org/T320281)  
The purpose of this pre-deployment QA is to confirm that events are logging as expected by the Metrics Platform. This instrumentation is deployed on [Test Wiki](https://test.wikipedia.org/wiki/Main_Page)



In [1]:
from wmfdata import hive, spark

### Goals
Verify that the instrumentation for tracking edit events is working successfully

###  Expectations
- The metrics should be comparable between event.editattemptstep and event.mediawiki_edit_attempt  
- Sampling is 100% which means all edit attempt events should be logged into the table  
- edit_attempt is a funnel schema. The number of events should be in this order:  
 init > ready Or loaded > firstChange or abort > saveIntent > saveAttempt > saveSuccess Or saveFailure

### Caveats 
- Ad blocker turned On
- Wikis and namespaces excluded from event logging : tba
- To do the QA, I had to use Superset for looking at sample data and HUE for the table structure, because Superset does not display `struct` columns well. Until Datahub provides the option for viewing sample data, I don't think we can use it as an effective tool for referencing datasets frequently.  

### Reference
- [Datahub](https://datahub.wikimedia.org/dataset/urn:li:dataset:(urn:li:dataPlatform:hive,event.mediawiki_edit_attempt,PROD)/Schema?is_lineage_mode=false&schemaFilter=)

### Observations
- All events are assigned a browser, os, device family   
- All discussiontools init events are assigned a section or page init_type  
- All events for a session id are in the correct funnel order  (loaded and ready events have the same count)
- All logged in users have a user_id  
### Issues
- Some logged-in users dont have the following fields populated - 
    - user_name (performer.name)
    - registration_dt (performer.registration_dt)
    - edit_count_bucket (performer.edit_count_bucket)
 

### Questions
- Detailed Documentation on the field definitions. A: will reside on Datahub
- where is the event.action captured? A: in the "name" field
- this will be updated from [gdoc](https://docs.google.com/document/d/1rJwTbbuOWuAjNyxlBLXWXVuRzO4f4vUjuEwJHw4ihsE/edit)

### QA Checks
- Measure the # of events for each step of an edit attempt action     
- Create new test data and track events created by my user id (pending)  
- Compare with editattemptstep (pending)   
- Check a few sample individual sessions, ordered by timestamp  
- Check overall data
    - meta data  
    - performer data  
    - user agent (map) data  
    - custom (bespoke) data - here you will find the properties of an event like timing, actions involved in an attempt to edit a page etc. which are specific to the instrumentation  
        - check VE events  
        - check discussion tools events  
        - check wikitext events  
  
        
        


### Notes


### Check the data in meta, performer, user agent, normalized host 

In [137]:
query1=spark.run('''
SELECT 
  date_format(dt, 'yyyy-MM-dd') AS date,
  meta.domain AS wiki,
  meta.id AS meta_id,
  meta.request_id as meta_req_id, 
  normalized_host.project as wiki_db, 
  normalized_host.project_family, 
  IF(performer.is_logged_in, "logged-in", "anonymous") AS user_log,
  performer.id AS user_id,
  performer.name AS user_name,
  performer.session_id,
  performer.registration_dt,
  performer.edit_count,
  performer.edit_count_bucket,
  performer.groups,
  user_agent_map['browser_family'] AS browser_family,  
  user_agent_map['browser_major'] AS browser_major,
  user_agent_map['os_family'] AS os_family,
  user_agent_map['os_minor'] AS os_minor ,
  user_agent_map['os_major'] AS os_major,
  user_agent_map['wmf_app_version'] AS wmf_app_version,
  user_agent_map['device_family'] AS device_family
  FROM event.mediawiki_edit_attempt
  ORDER BY date
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [138]:
query1.tail(10)

,date,wiki,meta_id,meta_req_id,wiki_db,project_family,user_log,user_id,user_name,session_id,...,edit_count,edit_count_bucket,groups,browser_family,browser_major,os_family,os_minor,os_major,wmf_app_version,device_family
2729,2022-11-03,test.wikipedia.org,0d2b29a2-2791-4b97-8a56-db77a6799b0d,3595f8d1-d0fb-4550-bea5-00ad42a19441,test,wikipedia,logged-in,54035,None,11b63f79445c7d450baa,...,1.0,None,None,Chrome,105,Mac OS X,15,10,-,Mac
2730,2022-11-03,test.wikipedia.org,679b1558-88cc-4545-9c7b-c444b4d1fefc,02dea918-af89-4afc-9959-2a9aa8967781,test,wikipedia,logged-in,54035,None,11b63f79445c7d450baa,...,1.0,None,None,Chrome,105,Mac OS X,15,10,-,Mac
2731,2022-11-04,test.wikipedia.org,fa1350b5-dd8b-4785-a625-e742e08a6585,bb7562e5-002a-4f64-9d2d-fc1de0f5a8d4,test,wikipedia,anonymous,0,None,92fed36697ed199a4bad,...,NaN,None,None,Chrome,81,Android,1,7,-,BM111
2732,2022-11-04,test.wikipedia.org,9e9723ac-6f07-4f03-8976-0edc2206dbd6,95a88008-5bf7-4e32-8489-9e79b1670c8d,test,wikipedia,anonymous,0,None,92fed36697ed199a4bad,...,NaN,None,None,Chrome,81,Android,1,7,-,BM111
2733,2022-11-04,test.wikipedia.org,846c0128-af10-4955-ae56-13e5fdf83efa,2877b861-a478-4658-aecf-9fea0b71e996,test,wikipedia,anonymous,0,None,92fed36697ed199a4bad,...,NaN,None,None,Chrome,81,Android,1,7,-,BM111
2734,2022-11-04,test.wikipedia.org,60062b84-f752-40a9-bead-ae40c793c3f2,c56bb927-697a-4a44-bb30-cd1a5f0e5606,test,wikipedia,anonymous,0,None,92fed36697ed199a4bad,...,NaN,None,None,Chrome,81,Android,1,7,-,BM111
2735,2022-11-04,test.wikipedia.org,d4004fc4-fa37-4ff2-904a-bad739c74728,95e476c0-0e35-48a1-9527-c450be8319a4,test,wikipedia,logged-in,55665,None,8d87251e2739718fe822,...,0.0,None,None,Chrome,107,Windows,-,10,-,Other
2736,2022-11-04,test.wikipedia.org,36eb3722-08f4-4342-be99-a18f6de57dc3,61aa8f46-8489-4433-970d-ac0e399ecbb7,test,wikipedia,logged-in,55665,None,8d87251e2739718fe822,...,0.0,None,None,Chrome,107,Windows,-,10,-,Other
2737,2022-11-04,test.wikipedia.org,6a5b8d6f-8d93-4395-ae23-e7ab0fc5f5bb,7bdbf292-9f2b-45f2-9b53-12b964fc07b1,test,wikipedia,logged-in,55665,None,8d87251e2739718fe822,...,0.0,None,None,Chrome,107,Windows,-,10,-,Other
2738,2022-11-04,test.wikipedia.org,4f24f932-06ee-4d8a-bc4f-b08031cac61d,658b3464-163d-4932-a87b-b0d31c0047f4,test,wikipedia,logged-in,55665,None,8d87251e2739718fe822,...,0.0,None,None,Chrome,107,Windows,-,10,-,Other


In [112]:
query1['wiki'].unique().tolist()

['test.wikipedia.org']

In [113]:
query1['project_family'].unique().tolist()

['wikipedia']

In [116]:
query1['edit_count_bucket'].unique().tolist()

[None]

In [117]:
query1['registration_dt'].unique().tolist()

[None]

In [134]:
query1['user_log'].value_counts()

logged-in    1975
anonymous     764
Name: user_log, dtype: int64

All logged in users have user_id

In [135]:
perf_query1=spark.run('''
SELECT 
  date_format(dt, 'yyyy-MM-dd') AS date,
  performer.id AS user_id,
  performer.name AS user_name
  FROM event.mediawiki_edit_attempt
  WHERE performer.is_logged_in
  AND performer.id=0
  ORDER BY date
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [136]:
perf_query1

,date,user_id,user_name


### All browser families

In [95]:
ua_query1['browser_family'].unique().tolist()

['Firefox',
 'Chrome',
 'Chrome Mobile',
 'Mobile Safari',
 'Safari',
 'Edge Mobile',
 'Edge',
 'Google',
 'Pale Moon',
 'Apple Mail',
 'Waterfox',
 'Opera',
 'Samsung Internet',
 'Firefox Mobile']

### Operating Systems

In [98]:
ua_query1['os_family'].unique().tolist()

['Windows', 'Mac OS X', 'Android', 'iOS', 'Linux', 'Chrome OS', 'Ubuntu']

### All Device families

In [99]:
ua_query1.device_family.unique()

array(['Other', 'Mac', 'Oppo CPH2269', 'iPhone', 'vivo 1906', 'MYA-L13',
       'Lenovo TB-X306X', 'LM-Q720', 'Samsung SM-G981U',
       'Samsung SM-M307F', 'Huawei DRA-L21', 'iPad', '5031G',
       'Samsung SM-G532F', 'Samsung SM-G950F', 'Oppo RMX3195', 'KB2005',
       'FP3', 'SM-T285', 'XiaoMi Mi 9T', 'Samsung SM-A315F', '22011119TI',
       'Lenovo TB-7104F', 'XiaoMi Redmi Note 8T', 'moto e20',
       'moto e5 plus', 'Oppo CPH2145', 'Nexus 5', 'Samsung SM-J400F',
       'XiaoMi Redmi 7A', 'Z33', 'Samsung SM-A125F', 'Samsung SM-J810Y',
       'Generic Smartphone', 'M2003J15SC', 'Samsung SM-G950U1', 'V2134',
       'Itel P651W', 'XiaoMi Redmi 8A Dual'], dtype=object)

In [22]:
date_query1=spark.run('''
SELECT 
  DISTINCT date_format(dt, 'yyyy-MM-dd') AS date
  FROM event.mediawiki_edit_attempt
  ORDER BY date
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [44]:
len(date_query1)

47

In [50]:
date_query1.min()

date    2022-09-13
dtype: object


In [51]:
date_query1.max()

date    2022-11-01
dtype: object


## Check the custom (bespoke) data

In [27]:
name_query1=spark.run('''
SELECT 
  DISTINCT name, count(1)
  FROM event.mediawiki_edit_attempt
  GROUP BY name
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


There are 26 distinct name types in the table

In [33]:
name_query1.sort_values(by=['count(1)','name'],ascending=False)

,name,count(1)
19,eas.ve.init,281
23,eas.ve.ready,261
25,eas.ve.loaded,261
9,eas.ve.abort,162
4,eas.mf.init,113
0,eas.mf.ready,109
22,eas.mf.loaded,109
3,eas.ve.first_change,108
21,eas.ve.save_intent,94
7,eas.dt.init,86


In [65]:
total_events=name_query1['count(1)'].sum()
total_events

2371

In [36]:
ve_events=spark.run('''
SELECT 
  DISTINCT name, count(1) AS event_count
  FROM event.mediawiki_edit_attempt
  WHERE name like 'eas.ve%'
  GROUP BY name
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


### Visual Editor edit events and types

In [38]:
ve_events.sort_values(by=['event_count','name'],ascending=False)

,name,event_count
4,eas.ve.init,281
6,eas.ve.ready,261
8,eas.ve.loaded,261
2,eas.ve.abort,162
0,eas.ve.first_change,108
5,eas.ve.save_intent,94
7,eas.ve.save_attempt,79
3,eas.ve.save_success,76
1,eas.ve.save_failure,3


In [64]:
ve_events['event_count'].sum()

1325

In [57]:
wikitext_events =spark.run('''
SELECT 
  DISTINCT name, count(1) AS event_count
  FROM event.mediawiki_edit_attempt
  WHERE name like 'eas.mf%'
  GROUP BY name
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


### wikitext events and types

In [58]:
wikitext_events.sort_values(by=['event_count','name'],ascending=False)

,name,event_count
1,eas.mf.init,113
0,eas.mf.ready,109
7,eas.mf.loaded,109
6,eas.mf.abort,56
4,eas.mf.first_change,52
2,eas.mf.save_intent,51
5,eas.mf.save_attempt,36
3,eas.mf.save_success,29


In [66]:
wikitext_events['event_count'].sum()

555

In [59]:
discussiontools_events =spark.run('''
SELECT 
  DISTINCT name, count(1) AS event_count
  FROM event.mediawiki_edit_attempt
  WHERE name like 'eas.dt%'
  GROUP BY name
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


### Reply tool events and types

In [60]:
discussiontools_events.sort_values(by=['event_count','name'],ascending=False)

,name,event_count
2,eas.dt.init,86
8,eas.dt.ready,80
0,eas.dt.loaded,80
6,eas.dt.first_change,61
7,eas.dt.save_intent,49
1,eas.dt.save_attempt,49
4,eas.dt.save_success,48
3,eas.dt.abort,37
5,eas.dt.save_failure,1


In [67]:
discussiontools_events['event_count'].sum()

491

In [84]:
discussiontools_events1 =spark.run('''
SELECT 
    custom_data['init_type'] as dt_type, count(1) AS event_count
  --IF(custom_data['init_type'] = "section", "new discussion tool", "reply tool") as dt_type, count(1) AS event_count
  --IF(custom_data.init_type = "section", "new discussion tool", "reply tool") as dt_type, count(1) AS event_count
  FROM event.mediawiki_edit_attempt
  WHERE name like 'eas.dt.init'
  GROUP BY dt_type
  --IF(custom_data.init_type = "section", "new discussion tool", "reply tool")
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


Reply Tool (section) vs New Discussion Tool (page) 
All dt init events are assigned a section or page init_type

In [85]:
discussiontools_events1

,dt_type,event_count
0,"(string, section)",36
1,"(string, page)",50


In [ ]:
## Check sample individual sessions 

In [118]:
check_event1 =spark.run('''
SELECT 
    *
  FROM event.mediawiki_edit_attempt
  WHERE performer.session_id='8d87251e2739718fe822'
  ORDER BY dt
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [119]:
check_event

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
0,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-04T13:39:10.255Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-04T13:39:15.395Z,...",eas.ve.init,"(SureshKumar0307/sandbox, None, 0, None, 2, No...","(None, 0, None, None, 55665, None, True, None,...","{'os_family': 'Windows', 'browser_major': '107...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,4,13
1,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-04T13:39:10.584Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-04T13:39:15.398Z,...",eas.ve.ready,"(SureshKumar0307/sandbox, None, 0, None, 2, No...","(None, 0, None, None, 55665, None, True, None,...","{'os_family': 'Windows', 'browser_major': '107...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,4,13
2,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-04T13:39:10.593Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-04T13:39:15.397Z,...",eas.ve.loaded,"(SureshKumar0307/sandbox, None, 0, None, 2, No...","(None, 0, None, None, 55665, None, True, None,...","{'os_family': 'Windows', 'browser_major': '107...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,4,13
3,None,"(None, mediawiki_js, desktop_browser)","{'abort_mechanism': ('string', 'navigate'), 'i...",2022-11-04T13:39:13.817Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-04T13:39:15.399Z,...",eas.ve.abort,"(SureshKumar0307/sandbox, None, 0, None, 2, No...","(None, 0, None, None, 55665, None, True, None,...","{'os_family': 'Windows', 'browser_major': '107...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,4,13


In [121]:
check_event2 =spark.run('''
SELECT 
    *
  FROM event.mediawiki_edit_attempt
  WHERE performer.session_id='ebbb63d0393cfd51c08a'
  ORDER BY dt
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


All events for a session id are in the correct funnel order

In [122]:
check_event2

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
0,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T03:54:56.662Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T03:55:25.338Z,...",eas.ve.init,"(Rose, None, 146951, None, 0, None, 548998, No...","(None, 132, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,3
1,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T03:54:57.231Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T03:55:25.359Z,...",eas.ve.loaded,"(Rose, None, 146951, None, 0, None, 548998, No...","(None, 132, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,3
2,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T03:54:57.231Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T03:55:25.358Z,...",eas.ve.ready,"(Rose, None, 146951, None, 0, None, 548998, No...","(None, 132, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,3
3,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T03:55:04.157Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T03:55:25.361Z,...",eas.ve.first_change,"(Rose, None, 146951, None, 0, None, 548998, No...","(None, 132, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,3
4,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T03:55:07.524Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T03:55:25.363Z,...",eas.ve.save_intent,"(Rose, None, 146951, None, 0, None, 548998, No...","(None, 132, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T05:00:06.504Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T05:00:22.053Z,...",eas.ve.loaded,"(Rose, None, 146951, None, 0, None, 549140, No...","(None, 156, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,5
158,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T05:00:11.925Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-03T05:00:22.056Z,...",eas.ve.first_change,"(Rose, None, 146951, None, 0, None, 549140, No...","(None, 156, None, None, 44768, None, True, Non...","{'os_family': 'Ubuntu', 'browser_major': '106'...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,3,5
159,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-03T05:00:14.354Z,"(None, None, None, {'user-agent': 'Mozilla